In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.stats import invgamma
import utils
import hashlib
from sklearn.mixture import GaussianMixture
from learn_diagonal_gauss_fs import bayesGMM_FS
from sklearn.metrics.cluster import adjusted_rand_score


In [2]:
N = 1000
K = 3
D = 10

seed = 41
seed = np.random.randint(1000, 2**16)
print(seed)
np.random.seed(seed)

54290


I'm using Normal-Inverse-chi2 according to following reparametrization of the normal-Inverse-Gamma hyperparameters. Ref: https://www.cs.ubc.ca/~murphyk/Papers/bayesGauss.pdf

m0_true is MU0\
k0_true is KAPPA0\
v0_true is 2 x ALPHA0\
s0_true is 2 x BETA0

In [3]:
m0_true = np.random.uniform(2, 8, (K, D))
k0_true = np.random.uniform(1, 1.5, K)
S0_true = np.random.uniform(0, 2, (K , D))
v0_true = np.random.randint(1, 5, K)

# m0_true = np.random.uniform(-1, 1, D)
# k0_true = np.random.uniform(0, 0.5)
# S0_true = np.random.uniform(0, 2, D)
# v0_true = np.random.randint(1, 3)

alpha_true = 1.0

In [4]:
[np.min(m0_true), np.max(m0_true)]

[2.2024147486994647, 7.985710545891205]

In [5]:
[np.min(S0_true), np.max(S0_true)]

[0.014869162996649088, 1.9152968215911814]

#### Sample mu and sigma

In [6]:
mu = np.zeros((K, D), float)
sigma = np.zeros((K, D), float)
for k in range(K):
    for j in range(D):
        mu[k, j] = norm.rvs(m0_true[k, j], S0_true[k, j]/(k0_true[k]*v0_true[k]))
        sigma[k, j] = invgamma.rvs(v0_true[k]/2, S0_true[k, j]/2)
        # mu[k, j] = np.random.normal(m0_true[j], S0_true[j]/(k0_true*v0_true))
        # sigma[k, j] = invgamma.rvs(v0_true/2, S0_true[j]/2)


In [7]:
[np.min(mu), np.max(mu)]

[1.9294829179228237, 7.986319589235545]

In [8]:
[np.min(sigma), np.max(sigma)]

[0.31414358025857064, 387.40430944741905]

#### Generate data

In [9]:
# mixing_probabilities = np.random.dirichlet([alpha_true]*K, size=1)[0]
# z_true = np.random.choice(K, p=mixing_probabilities, size=N)

z_true = np.random.randint(0, K, N)

X = np.zeros((N, D))

for i in range(N):
    for j in range(D):
        for k in range(K):
            if z_true[i] == k:
                X[i, j] = np.random.normal(mu[k, j], sigma[k, j])



In [10]:
[np.min(X), np.max(X)]

[-923.5067266414741, 926.7937227738677]

In [11]:
# heatmap = sns.heatmap(X[np.argsort(z_true)])

In [12]:
### Save data and labels

filepath = f"dataHyperFS/{N}_{K}_{D}_{seed}_{hashlib.sha256(X.tobytes()).hexdigest()[:5]}"
print(utils.saveData(f"{filepath}.csv", X, remark="Data"))
print(utils.saveData(f"{filepath}.labels", z_true, remark="Labels"))

dataHyperFS/1000_3_10_54290_dec7d.csv
dataHyperFS/1000_3_10_54290_dec7d.labels


In [13]:
# heatmap = sns.heatmap(X[np.argsort(z_true)])

### Run Scikit-GMM before adding noise

In [14]:
## Run scikit GMM with no noisy data

training_runs_EM = 100
EM_gmm = GaussianMixture(n_components=K, n_init=training_runs_EM)
EM_gmm.fit(X)
print(f"BIC: {EM_gmm.bic(X)}\nK: {K}")
z_pred_EM = EM_gmm.predict(X)

print("ARI:", round(adjusted_rand_score(z_true, z_pred_EM),3))

BIC: 57244.702242407206
K: 3
ARI: 0.536


## Add noise

In [15]:
nd = 7 # number of noisy features

In [16]:
isLOCAL = False # global or local

true_features = np.ones((K, D), int)

if not isLOCAL:
    true_features[:, :nd] = np.zeros((K, nd), int)
else:
    for j in range(nd):
        rand_cluster = np.random.randint(0, 2, K)
        true_features[:, j] = rand_cluster

print(true_features)

X_noisy = X.copy()

X_mean = np.mean(X)
X_var = np.var(X)
for k in range(K):
    for j in range(D):
        if true_features[k][j] == 0:
            for i in range(N):
                if z_true[i] == k :
                    X_noisy[i, j] = np.random.normal(10*X_mean, 0.1*X_var)


[[0 0 0 0 0 0 0 1 1 1]
 [0 0 0 0 0 0 0 1 1 1]
 [0 0 0 0 0 0 0 1 1 1]]


In [17]:
# heatmap = sns.heatmap(X[np.argsort(z_true)])

In [18]:
### Save data

filepath = f"dataHyperFS/{N}_{K}_{D}_{seed}_{hashlib.sha256(X_noisy.tobytes()).hexdigest()[:5]}_noise_{nd}"
print(utils.saveData(f"{filepath}.csv", X_noisy, remark="Data"))
print(utils.saveData(f"{filepath}.features", true_features, remark="Data"))
print(utils.saveData(f"{filepath}.labels", z_true, remark="Labels"))

dataHyperFS/1000_3_10_54290_932a0_noise_7.csv
dataHyperFS/1000_3_10_54290_932a0_noise_7.features
dataHyperFS/1000_3_10_54290_932a0_noise_7.labels


In [19]:
## Run scikit GMM with noisy Data

training_runs_EM = 100
EM_gmm_noisy = GaussianMixture(n_components=K, n_init=training_runs_EM)
EM_gmm_noisy.fit(X_noisy[:, -3:])
print(f"BIC: {EM_gmm_noisy.bic(X_noisy[:, -3:])}\nK: {K}")
z_pred_EM_noisy = EM_gmm_noisy.predict(X_noisy[:, -3:])

print("ARI:", round(adjusted_rand_score(z_true, z_pred_EM_noisy),3))

BIC: 21565.709609107515
K: 3
ARI: 0.358


## Run our model

In [20]:
### Set the params

alpha = 1.0 
m_0 = np.zeros(D)
k_0 = 0.03 
v_0 = D + 3
S_0 = 0.3*v_0*np.ones(D)

K_initial = K
iterations = 30
training_runs = 4

#### No Feature Selection

In [21]:
prior = utils.NIchi2(m_0, k_0, v_0, S_0)

Models = []
Results = []
    
FS = False

initial_ass_seed = 25
np.random.seed(initial_ass_seed)
for run in range(training_runs):
    print(f"\nRun:  {run+1}")
    starting_assignments = []

    while len(set(starting_assignments)) != K_initial:
        starting_assignments = np.random.randint(0, K_initial, N)
    
    bayesgmm = bayesGMM_FS(X_noisy, prior, alpha, starting_assignments, FS)
    gmmResult = bayesgmm.gibbs_sampler(iterations, run, toPrint=True, savePosterior=False, trueAssignments=z_true)
    
    Models.append(bayesgmm)
    Results.append(gmmResult)



Run:  1
Initial features:
[[1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]]
run: 1, iteration:0, K:3, posterior:-60905.863, ARI: -0.0

run: 1, iteration:1, K:3, posterior:-60400.11, ARI: 0.0569889086042997
0/30               
run: 1, iteration:2, K:3, posterior:-58995.873, ARI: 0.300715463593699
1/30               
run: 1, iteration:3, K:3, posterior:-58859.053, ARI: 0.33241188216937184
2/30               
run: 1, iteration:4, K:3, posterior:-58884.111, ARI: 0.3287775325914989
3/30               
run: 1, iteration:5, K:3, posterior:-58880.886, ARI: 0.33154890041016166
4/30               
run: 1, iteration:6, K:3, posterior:-58836.949, ARI: 0.327527058545857
5/30               
run: 1, iteration:7, K:3, posterior:-58869.239, ARI: 0.30963192566248293
6/30               
run: 1, iteration:8, K:3, posterior:-58860.579, ARI: 0.33135446718630956
7/30               
run: 1, iteration:9, K:3, posterior:-58845.982, ARI: 0.3215659160652319
8/30               
run: 1, iterati

In [22]:
ll = sorted([[round(adjusted_rand_score(z_true, model.z_map),3), int(model.BIC) ,int(model.log_max_post), model.iter_map] for model in Models])[::-1]
print("ARI BIC LogMaxPost MAP_iter")
for i in ll:
    print(*i)
    
# choosing the best model with lowest BIC
least_BIC = 1*np.inf
for model in Models:
    if model.BIC < least_BIC:
        bestModel = model
        least_BIC = model.BIC

    if D == 2:
        colors = np.array([x for x in "rgcmykbgrbgcmykbgrcmykbgrcmyk"])
        plt.scatter(X[:, 0], X[:, 1], color=colors[model.z_map].tolist())
        plt.show()
        plt.close()

bayes_assignments = bestModel.z_map

ARI BIC LogMaxPost MAP_iter
0.858 115385 -57677 30
0.85 115393 -57681 29
0.845 115388 -57679 27
0.435 117238 -58604 16


In [23]:
bestModel.clusters.counts

array([332, 314, 354])

#### With Feature Selection

In [27]:
prior = utils.NIchi2(m_0, k_0, v_0, S_0)

Models = []
Results = []
    
FS = True

features = np.ones((K_initial, D), int)

initial_ass_seed = 25
np.random.seed(initial_ass_seed)
for run in range(training_runs):
    print(f"\nRun:  {run+1}")
    starting_assignments = []

    while len(set(starting_assignments)) != K_initial:
        starting_assignments = np.random.randint(0, K_initial, N)
    
    bayesgmm = bayesGMM_FS(X_noisy, prior, alpha, starting_assignments, FS, features)
    gmmResult = bayesgmm.gibbs_sampler(iterations, run, toPrint=True, savePosterior=False, trueAssignments=z_true)
    
    Models.append(bayesgmm)
    Results.append(gmmResult)



Run:  1
Initial features:
[[1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]]
run: 1, iteration:0, K:3, posterior:-60905.863, ARI: -0.0

run: 1, iteration:1, K:3, posterior:-60400.11, ARI: 0.0569889086042997
features:
 [[1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]]
0/30               
run: 1, iteration:2, K:3, posterior:-58995.873, ARI: 0.300715463593699
features:
 [[1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]]
1/30               
run: 1, iteration:3, K:3, posterior:-58859.053, ARI: 0.33241188216937184
features:
 [[1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]]
2/30               
run: 1, iteration:4, K:3, posterior:-58844.874, ARI: 0.33423571054367257
features:
 [[1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]]
3/30               
run: 1, iteration:5, K:3, posterior:-58873.732, ARI: 0.3372355981294535
features:
 [[1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 0]]

In [29]:
ll = sorted([[round(adjusted_rand_score(z_true, model.z_map),3), int(model.BIC) ,int(model.log_max_post), model.iter_map] for model in Models])[::-1]
print("ARI BIC LogMaxPost MAP_iter")
for i in ll:
    print(*i)
    
# choosing the best model with lowest BIC
least_BIC = 1*np.inf
for model in Models:
    if model.BIC < least_BIC:
        bestModel = model
        least_BIC = model.BIC

bayes_assignments = bestModel.z_map

ARI BIC LogMaxPost MAP_iter
0.488 117548 -58767 19
0.463 117758 -58872 13
0.461 117301 -58643 30
0.457 117746 -58866 16


In [ ]:
# heatmap = sns.heatmap(X[np.argsort(bayes_assignments)])

In [ ]:
[np.count_nonzero(z_true==i) for i in range(K)]

In [ ]:
bestModel